In [1]:
from flask import Flask,Response,jsonify,request
import json
from bson.objectid import ObjectId

In [2]:
app=Flask(__name__)

In [3]:
#! pip install pymongo

In [4]:
import pymongo

In [5]:
try: 
    mongo=pymongo.MongoClient(host="localhost",port=27017,serverSelectionTimeoutMS=1000)
    # Company is the name of the database
    db=mongo.company
    # triggers exception, if cannot be connected to the db.
    mongo.server_info()
except:
    print("Error - Cannot connect to DB")

In [6]:
@app.route("/users",methods=['POST'])
def create_user():
    try:
        user={'name':request.form['name'],'lastname': request.form['last']}
        
        # users is the name of the collection of the database named company
        dbResponse=db.users.insert_one(user)
        print(dbResponse.inserted_id)
    except Exception as ex:
        print(ex)
    return Response(response=json.dumps({"message":"user created", "id":f"{dbResponse.inserted_id}"}),
                    status=200,mimetype="application/json")

In [7]:
@app.route("/users",methods=['GET'])
def get_some_users():
    try:
        data= db.users.find()
        return Response(response=json.dumps({"message":f"{list(data)}"}),
                    status=200,mimetype="application/json")
    except Exception as ex:
        print(ex)
        return Response(response=json.dumps({"message":"Cannot Read Users"}),
                    status=500,mimetype="application/json")

In [8]:
@app.route("/users/<id>",methods=["PUT"]) 
def update_user(id):
    try:
        dbResponse=db.users.update_one(
            {"_id":ObjectId(id)},
            {"$set":{"name":request.form["name"]}})
        
#         for attr in dir(dbResponse):
#             print(f"************{attr}***********")
        
        if dbResponse.modified_count:
            return Response(response=json.dumps({"message":"User Updated"}),
                        status=200,mimetype="application/json")
        else: 
            return Response(response=json.dumps({"message":"Nothing to Update"}),
                        status=200,mimetype="application/json")
    except Exception as ex:
        print("*"*50)
        print(ex)
        print("*"*50)
        return Response(response=json.dumps({"message":"Sorry Cannot Update"}),
                    status=200,mimetype="application/json")

In [9]:
@app.route("/users/<id>",methods=["DELETE"])
def delete_user(id):
    try:
        dbResponse=db.users.delete_one({"_id":ObjectId(id)})
#         for attr in dir(dbResponse):
#         print(f"************{attr}***********")
        if dbResponse.deleted_count:
            return Response(response=json.dumps({"message":"User Deleted","id":f"{id}"}),
                        status=200,mimetype="application/json")
        else: 
            return Response(response=json.dumps({"message":"User Not Found","id":f"{id}"}),
                        status=200,mimetype="application/json")

    except Exception as ex:
        print("*"*50)
        print(ex)
        print("*"*50)
        return Response(response=json.dumps({"message":"Sorry Cannot delete"}),
                    status=500,mimetype="application/json")
        

In [10]:
if __name__== "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Sep/2020 18:01:20] "DELETE /users/5f55f6c24e8c1e0361919c45 HTTP/1.1" 200 -
